In [8]:
import numpy as np
import torch
from smplx import MANO
import mujoco
import numpy as np
import sys
sys.path.append("../utils")
from mujoco_parser import MuJoCoParserClass
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

xml_path = '../asset/object/floor_sky.xml'
env = MuJoCoParserClass(name='',rel_xml_path=xml_path,VERBOSE=False)

MuJoCo version:[3.1.6]


In [9]:
comp_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
left_mano_path = 'data/mano/MANO_LEFT.pkl'
right_mano_path = 'data/mano/MANO_RIGHT.pkl'

left_model = MANO(
    model_path=left_mano_path,
    is_rhand=False,
    use_pca=False,
    flat_hand_mean=True
).to(comp_device)

right_model = MANO(
    model_path=right_mano_path,
    is_rhand=True,
    use_pca=False,
    flat_hand_mean=True
).to(comp_device)


In [10]:
data_number = 493
data_path = '../data/both57m/%d/data.npz'%(data_number)
data = np.load(data_path)
seq_len = data['seq_len']

hand_pose = torch.Tensor(data['hand_pose'].reshape((seq_len,-1))).to(comp_device)
left_hand_pose = hand_pose[:,:45]
right_hand_pose = hand_pose[:,45:]

print(left_hand_pose.shape)
print(right_hand_pose.shape)

torch.Size([150, 45])
torch.Size([150, 45])


In [11]:
left_joint_pos_list = []
right_joint_pos_list = []
left_extra_joints_idxs = left_model.vertex_joint_selector.extra_joints_idxs
right_extra_joints_idxs = right_model.vertex_joint_selector.extra_joints_idxs
finger_idxs = [0,1,2,3,5,6,7,9,10,11,13,14,15,17,18,19]
fingertip_idxs = [20,4,8,16,12]

for i in range(seq_len):
    left_output = left_model(hand_pose=left_hand_pose[i].unsqueeze(0),return_verts=True)
    left_finger_pos_output = left_output.joints.detach().cpu().numpy().squeeze()
    left_fingertip_pos_output = left_output.vertices[:,left_extra_joints_idxs,:].detach().cpu().numpy().squeeze()
    left_full_joint_pos = np.zeros((21,3))
    left_full_joint_pos[finger_idxs] = left_finger_pos_output
    left_full_joint_pos[fingertip_idxs] = left_fingertip_pos_output
    left_joint_pos_list.append(left_full_joint_pos)

    right_output = right_model(hand_pose=right_hand_pose[i].unsqueeze(0),return_verts=True)
    right_finger_pos_output = right_output.joints.detach().cpu().numpy().squeeze()
    right_fingertip_pos_output = right_output.vertices[:,right_extra_joints_idxs,:].detach().cpu().numpy().squeeze()
    right_full_joint_pos = np.zeros((21,3))
    right_full_joint_pos[finger_idxs] = right_finger_pos_output
    right_full_joint_pos[fingertip_idxs] = right_fingertip_pos_output
    right_joint_pos_list.append(right_full_joint_pos)

left_joint_pos_array = np.array(left_joint_pos_list)
right_joint_pos_array = np.array(right_joint_pos_list)
print("Left Joint Position shape: ", left_joint_pos_array.shape)
print("Right Joint Position shape: ", right_joint_pos_array.shape)

Left Joint Position shape:  (150, 21, 3)
Right Joint Position shape:  (150, 21, 3)


In [12]:
hand_connections = [
    [0,1],[1,2],[2,3],[3,4],
    [0,5],[5,6],[6,7],[7,8],
    [0,9],[9,10],[10,11],[11,12],
    [0,13],[13,14],[14,15],[15,16],
    [0,17],[17,18],[18,19],[19,20]
]

all_connections  = np.array(hand_connections)

In [13]:
import time
env.init_viewer(viewer_title='',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=180,distance=2.6,elevation=-25,lookat=[0.0,0.0,0.83])
env.reset()

tick = 0
while tick < seq_len and env.is_viewer_alive():
    left_joint_position = left_joint_pos_array[tick] + np.array([0.0,-0.3,0.3])
    right_joint_position = right_joint_pos_array[tick] + np.array([0.0,0.3,0.3])
    for i in range(left_joint_position.shape[0]):
        env.plot_sphere(p=left_joint_position[i],r=0.01,rgba=[1,0,0,1])
        env.plot_sphere(p=right_joint_position[i],r=0.01,rgba=[1,0,0,1])

    for j in range(np.shape(all_connections)[0]):
        a = all_connections[j][0]
        b = all_connections[j][1]
        env.plot_cylinder_fr2to(p_fr=left_joint_position[a],p_to=left_joint_position[b],r=0.003,rgba=[0.5,0.5,0.5,1])
        env.plot_cylinder_fr2to(p_fr=right_joint_position[a],p_to=right_joint_position[b],r=0.003,rgba=[0.5,0.5,0.5,1])

    env.render()
    env.plot_T(p=np.array([0.1,-0.3,0.2]),R=np.eye(3),label="Left Hand",PLOT_AXIS=False)
    env.plot_T(p=np.array([0.1,0.3,0.2]),R=np.eye(3),label="Right Hand",PLOT_AXIS=False)
    env.plot_T(p=np.zeros(3),R=np.eye(3),label="Tick: [%d/%d]"%(tick,seq_len),PLOT_AXIS=False)
    
    tick += 1
    if tick == seq_len: tick = 0
    time.sleep(1/30)

env.close_viewer()

Pressed ESC
Quitting.
